# Default Configs and Logging Configuration in SageMakerCore

---
## Introductions

In this notebook, we will walkthrough the setup and usage of default configs in the SageMakerCore SDK. Additionally, this notebook contains a section with the steps required for configuring logging levels to assist in the debugging of issues that arise while using the SDK.



### Default Configs

Default Configs is a feature provided by the SageMakerCore SDK to assist users in defining default values to be auto populated into the AWS API Request parameters. For example, if a user/admin wants all of their AWS Resources to use a specific VPC Config during creation this can be defined in the Default Configs Configs. Default Configs supports:
1. GlobalDefaults - default values applied across SageMaker API calls
2. Resource Specific Defaults - defaults applied only when creating a specific resource

An Example of the strucuture of the Default Configs Config is below:

```json
{
    "SchemaVesion": "1.0",
    "SageMaker": {
        "PythonSDK": {
            "Resources": {
                "GlobalDefaults": {
                    "vpc_config": {
                        "security_group_ids": [
                            "sg-xxxxxxxxxxxxxxxxx" // Replace with security group id
                        ],
                        "subnets": [
                            "subnet-xxxxxxxxxxxxxxxxx", // Replace with subnet id
                            "subnet-xxxxxxxxxxxxxxxxx" // Replace with subnet id
                        ]
                    }
                    // ...
                },
                "TrainingJob": {
                    "role_arn": "arn:aws:xxxxxxxxxxx:role/xxxxx", // Replace with role arn
                    "output_data_config": {
                        "s3_output_path": "s3://xxxxxxxxxxx", // Replace with S3 URI
                    },
                    // ...
                }
            }
        }
    }
}
```

### Logging Levels

To assist in debugging issues originating within the SDK, SageMakerCore provides a simple utility method - `configure_logging()`

To set the logging level users have 2 options:
1. Pass a string parameter to utility method with log level they desire - `configure_logging("DEBUG")`
2. Set the `LOG_LEVEL=INFO` environment variable and call `configure_logging()` without a parameter


In a later section in this notebook, we will walk through an example of how these options would look like in practice for a user.

## Pre-Requisites

### Install Latest SageMakerCore
All SageMakerCore beta distributions will be released to a private s3 bucket. After being allowlisted, run the cells below to install the latest version of SageMakerCore from `s3://sagemaker-core-beta-artifacts/sagemaker_core-latest.tar.gz`

Ensure you are using a kernel with python version >=3.8

In [2]:
# Uninstall previous version of sagemaker_core and restart kernel
!pip uninstall sagemaker-core -y

In [3]:
# Install the latest version of sagemaker_core

!pip install sagemaker-core --upgrade

  Obtaining dependency information for sagemaker-core from https://files.pythonhosted.org/packages/4b/b0/418d1a2d75167fb08e0bf1be481a14a38586666e40c287803dbdbb8a8e89/sagemaker_core-1.0.47-py3-none-any.whl.metadata
  Obtaining dependency information for boto3<2.0.0,>=1.35.36 from https://files.pythonhosted.org/packages/97/0e/f0cb4f71c40ba07e6ed5b47699a737a080d3c4f4b7b26657d5671de48621/boto3-1.40.1-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic<3.0.0,>=2.0.0 from https://files.pythonhosted.org/packages/6a/c0/ec2b1c8712ca690e5d61979dee872603e92b8a32f94cc1b72d53beab008a/pydantic-2.11.7-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 3.4 MB/s eta 0:00:00
  Obtaining dependency information for rich<15.0.0,>=13.0.0 from https://files.pythonhosted.org/packages/e3/30/3c4d035596d3cf444529e0b2953ad0466f6049528a879d27534700580395/rich-14.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for mock<5.0,>4.0 from https://fi

In [4]:
# Check the version of sagemaker_core
!pip show -v sagemaker-core

Name: sagemaker-core
Version: 1.0.47
Summary: An python package for sagemaker core functionalities
Home-page: 
Author: 
Author-email: AWS <sagemaker-interests@amazon.com>
License: 
Location: /Users/nargokul/workspace/sagemaker-core-2/venv/lib/python3.10/site-packages
Requires: boto3, importlib-metadata, jsonschema, mock, platformdirs, pydantic, PyYAML, rich
Required-by: 
Metadata-Version: 2.4
Installer: pip
Classifiers:
  Development Status :: 3 - Alpha
  Intended Audience :: Developers
  License :: OSI Approved :: Apache Software License
  Programming Language :: Python :: 3.8
  Programming Language :: Python :: 3.9
  Programming Language :: Python :: 3.10
  Programming Language :: Python :: 3.11
  Programming Language :: Python :: 3.12
Entry-points:
Project-URLs:
  Repository, https://github.com/aws/sagemaker-core.git


### Install Additional Packages

In [5]:
# Install additionall packages

!pip install -U scikit-learn pandas boto3

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/fd/52/3056b6adb1ac58a0bc335fc2ed2fcf599974d908855e8cb0ca55f797593c/scikit_learn-1.7.1-cp310-cp310-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/80/bf/7938dddc5f01e18e573dcfb0f1b8c9357d9b5fa6ffdee6e605b92efbdff2/pandas-2.3.1-cp310-cp310-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.9 MB/s eta 0:00:00
  Obtaining dependency information for numpy>=1.22.0 from https://files.pythonhosted.org/packages/fd/77/dc2fcfc66943c6410e2bf598062f5959372735ffda175b39906d54f02349/numpy-2.2.6-cp310-cp310-macosx_14_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.2 MB/s eta 0:00:00
  Obtaining dependency information for scipy>=1.8.0 from https://files.pythonhosted.org/packages/ea/b1/4deb37252311c1acff7f101f6453f0440794f51b6eacb1aad4459a134081/scipy-1.15.3-cp3

### Setup

Let's start by specifying:
- AWS region.
- The IAM role arn used to give learning and hosting access to your data. Ensure your enviornment has AWS Credentials configured.
- The S3 bucket that you want to use for storing training and model data.

## Enable Rich Logging

In [6]:
from sagemaker_core import enable_rich_logging

enable_rich_logging()

In [7]:
from sagemaker_core.helper.session_helper import Session, get_execution_role
from rich import print

# Get region, role, bucket

sagemaker_session = Session()
region = sagemaker_session.boto_region_name
role = get_execution_role()
bucket = sagemaker_session.default_bucket()
print(role)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5                                                                                             │
│    6 sagemaker_session = Session()                                                               │
│    7 region = sagemaker_session.boto_region_name                                                 │
│ ❱  8 role = get_execution_role()                                                                 │
│    9 bucket = sagemaker_session.default_bucket()                                                 │
│   10 print(role)                                                                                 │
│   11                                                                                             │
│                                                                                                  │
│ /Users/nargokul/workspace/sagemaker-core-2/sagemaker-core-2/src/sagemaker_core/helper/session_he │
│ lper.py:726 in get_execution_role                                                                │
│                                                                                                  │
│   723 │   """                                                                                    │
│   724 │   if not sagemaker_session:                                                              │
│   725 │   │   sagemaker_session = Session()                                                      │
│ ❱ 726 │   arn = sagemaker_session.get_caller_identity_arn()                                      │
│   727 │                                                                                          │
│   728 │   if ":role/" in arn:                                                                    │
│   729 │   │   return arn                                                                         │
│                                                                                                  │
│ /Users/nargokul/workspace/sagemaker-core-2/sagemaker-core-2/src/sagemaker_core/helper/session_he │
│ lper.py:271 in get_caller_identity_arn                                                           │
│                                                                                                  │
│   268 │   │   │   "sts",                                                                         │
│   269 │   │   │   region_name=self.boto_region_name,                                             │
│   270 │   │   │   endpoint_url=sts_regional_endpoint(self.boto_region_name),                     │
│ ❱ 271 │   │   ).get_caller_identity()["Arn"]                                                     │
│   272 │   │                                                                                      │
│   273 │   │   role = re.sub(r"^(.+)sts::(\d+):assumed-role/(.+?)/.*$", r"\1iam::\2:role/\3", a   │
│   274                                                                                            │
│                                                                                                  │
│ /Users/nargokul/workspace/sagemaker-core-2/venv/lib/python3.10/site-packages/botocore/client.py: │
│ 601 in _api_call                                                                                 │
│                                                                                                  │
│    598 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    599 │   │   │   │   )                                                                         │
│    600 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  601 │   │   │   return self._make_api_call(operation_nam

### Load and Prepare Dataset
For this example, we will be using the IRIS data set from `sklearn.datasets` to train our XGBoost container.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import pandas as pd

# Get IRIS Data

iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

In [ ]:
import os

# Prepare Data

os.makedirs('./data', exist_ok=True)

iris_df = iris_df[['target'] + [col for col in iris_df.columns if col != 'target']]

train_data, test_data = train_test_split(iris_df, test_size=0.2, random_state=42)

train_data.to_csv('./data/train.csv', index=False, header=False)

### Upload Data to S3
In this step, we will upload the train and test data to the S3 bucket configured earlier using `sagemaker_session.default_bucket()`

In [ ]:
# Upload Data

prefix = "DEMO-scikit-iris"
TRAIN_DATA = "train.csv"
DATA_DIRECTORY = "data"

train_input = sagemaker_session.upload_data(
    DATA_DIRECTORY, bucket=bucket, key_prefix="{}/{}".format(prefix, DATA_DIRECTORY)
)

s3_input_path = "s3://{}/{}/data/{}".format(bucket, prefix, TRAIN_DATA)
s3_output_path = "s3://{}/{}/output".format(bucket, prefix)

print(s3_input_path)
print(s3_output_path)

### Fetch the XGBoost Image URI
In this step, we will fetch the XGBoost Image URI we will use as an input parameter when creating an AWS TrainingJob

In [ ]:
# Image name is hardcoded here
# Image name can be programatically got by using sagemaker package and calling image_uris.retrieve
# Since that is a high level abstraction that has multiple dependencies, the image URIs functionalities will live in sagemaker (V2)

image = "433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest"

## Default Configs

### Create Default Configs JSON

In order for SageMakerCore to pick up the Default Configs Configs to populate API calls, we first must create the json config file and set the `SAGEMAKER_CORE_ADMIN_CONFIG_OVERRIDE` enviornment variable.

Below we will create the config file at `data/defaults.json` and assign this path to the `SAGEMAKER_CORE_ADMIN_CONFIG_OVERRIDE` enviornment variable.

In [ ]:
import os
import json

DEFAULTS_CONTENT = {
    "SchemaVesion": "1.0",
    "SageMaker": {
        "PythonSDK": {
            "Resources": {
                "GlobalDefaults": {
                    "vpc_config": {
                        "security_group_ids": [
                            "sg-xxxxxxxxxxxxxxxxx" # Replace with security group id
                        ],
                        "subnets": [
                            "subnet-xxxxxxxxxxxxxxxxx", # Replace with subnet id
                            "subnet-xxxxxxxxxxxxxxxxx" # Replace with subnet id
                        ]
                    }
                },
                "TrainingJob": {
                    "role_arn": role,
                    "output_data_config": {
                        "s3_output_path": s3_output_path
                    },
            
                }
            }
        }
    }
}

path_to_defaults = os.path.join(DATA_DIRECTORY, "defaults.json")
with open(os.path.join(DATA_DIRECTORY, "defaults.json"), "w") as f:
    json.dump(DEFAULTS_CONTENT, f, indent=4)  

In [ ]:
import os
# Setting path of Config file in environment variable 
os.environ['SAGEMAKER_CORE_ADMIN_CONFIG_OVERRIDE'] = path_to_defaults

### Using GlobalDefaults
In the below example,  a `Cluster` resource will be created using the `vpc_config` defined under the `SageMaker.PythonSDK.Resources.GlobalDefaults`.

In [ ]:
import time
from sagemaker_core.resources import Cluster
from sagemaker_core.shapes import ClusterInstanceGroupSpecification, ClusterLifeCycleConfig
    
cluster_name_v3 = 'xgboost-cluster-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

# Use vpc_config from Default Configs JSON config file under the SageMaker.PythonSDK.Resources.GlobalDefaults key
cluster = Cluster.create(
    cluster_name=cluster_name_v3,
    instance_groups=[
        ClusterInstanceGroupSpecification(
            instance_count=1, 
            instance_group_name="instance-group-11",
            instance_type="ml.m5.4xlarge",
            life_cycle_config=ClusterLifeCycleConfig(source_s3_uri=s3_input_path, on_create="dothis"),
            execution_role=role
        )
    ]
)

In [ ]:
cluster.wait_for_status("InService")

### Using Resource Defaults
In the below example, a `TrainingJob` resource will be created using the `role` and `output_data_config` defined under the `SageMaker.Python.Resources.TrainingJob` key. 

Note: Because `TrainingJob` also excepts a `vpc_config` parameter, the `vpc_config` parameter will be populated from the `GlobalDefaults`

In [ ]:
import time
from sagemaker_core.resources import TrainingJob
from sagemaker_core.shapes import  AlgorithmSpecification, Channel, DataSource, S3DataSource, ResourceConfig, StoppingCondition

job_name_v3 = 'xgboost-iris-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

# Use role and output_data_config from Default Configs JSON config file under the SageMaker.PythonSDK.Resources.TrainingJob key
# Use vpc_config from Default Configs JSON config file under the SageMaker.PythonSDK.Resources.GlobalDefaults key

training_job = TrainingJob.create(
    training_job_name=job_name_v3,
    hyper_parameters={
        'objective': 'multi:softmax',
        'num_class': '3',
        'num_round': '10',
        'eval_metric': 'merror'
    },
    algorithm_specification=AlgorithmSpecification(
        training_image=image,
        training_input_mode='File'
    ),
    input_data_config=[
        Channel(
            channel_name='train',
            content_type='csv',
            compression_type='None',
            record_wrapper_type='None',
            data_source=DataSource(
                s3_data_source=S3DataSource(
                    s3_data_type='S3Prefix',
                    s3_uri=s3_input_path,
                    s3_data_distribution_type='FullyReplicated'
                )
            )
        )
    ],
    resource_config=ResourceConfig(
        instance_type='ml.m4.xlarge',
        instance_count=1,
        volume_size_in_gb=30
    ),
    stopping_condition=StoppingCondition(
        max_runtime_in_seconds=600
    )
)

## Configure Logging Levels

Below are 2 examples of how a SageMakerCore user could configure the logging level of the SDK to assist with debugging.

To set the logging level users have 2 options:
1. Pass a string parameter to utility method with log level they desire - `configure_logging("DEBUG")`
2. Set the `LOG_LEVEL=INFO` environment variable and call `configure_logging()` without a parameter

### Configure Logging with Parameter

In [ ]:
# Setting log_level to DEBUG using configure_logging with string parameter 
from sagemaker_core.main.utils import configure_logging

configure_logging('DEBUG')

In [ ]:
# Get TrainingJob with DEBUG log_level
from sagemaker_core.resources import TrainingJob

training_job = TrainingJob.get(job_name_v3)

### Configure Logging with Enviornment Variable

In [ ]:
# Setting log_level to INFO using env variable
!export LOG_LEVEL=INFO

configure_logging()

In [ ]:
# List TrainingJobs with INFO log_level
training_job = TrainingJob.get(job_name_v3)

## Delete All SageMaker Resources
The following code block will call the delete() method for any SageMaker Core Resources created during the execution of this notebook which were assigned to local or global variables. If you created any additional deleteable resources without assigning the returning object to a unique variable, you will need to delete the resource manually by doing something like:

```python
resource = Resource.get("resource-name")
resource.delete()
```


In [ ]:
# Delete any sagemaker core resource objects created in this notebook
def delete_all_sagemaker_resources():
    all_objects = list(locals().values()) + list(globals().values())
    deletable_objects = [obj for obj in all_objects if hasattr(obj, 'delete') and obj.__class__.__module__ == 'sagemaker_core.main.resources']
    
    for obj in deletable_objects:
        obj.delete()
        
delete_all_sagemaker_resources()